In [ ]:
def fit_lda(training_features, training_labels):
    m_1 = np.array([0,0])
    m1 = np.array([0,0])
    covmat_1 = np.matrix([[0,0], [0,0]])
    covmat1 = np.matrix([[0,0], [0,0]])
    N = len(training_features)
    
    ## mu-matrix
    for i in range(N):
        if training_labels[i] == -1:
            m_1 = m_1 + training_features[i]
        else:
            m1 = m1 + training_features[i]
    m_1 = m_1*(1/sum(training_labels == -1))
    m1 = m1*(1/sum(training_labels == 1))
    mu = np.matrix([m_1, m1])
    
    ## covmat 
    for i in range(N):
        if training_labels[i] == -1:
            temp1 = np.matrix(training_features[i]-mu[0])
            covmat_1 = covmat_1 + np.transpose(temp1).dot(temp1)
        else:
            temp1 = np.matrix(training_features[i]-mu[1])
            covmat1 = covmat1 + np.transpose(temp1).dot(temp1)
    covmat = covmat_1 + covmat1
    covmat = covmat/N
    
    ## p, what is p, what are the 'two priors'?
    p = 0
    
    return mu, covmat, p
training_features = features2d(X_train)
training_labels = correct_labels(y_train)
mu, covmat, p = fit_lda(training_features, training_labels)

In [ ]:
import math ## for log-function
def predict_lda(mu, covmat, p, test_features, test_labels):
    N1 = sum(test_labels == 1)
    N_1 = sum(test_labels == -1)
    N = N1+N_1
    predicted_labels = []
    
    beta = np.linalg.inv(covmat).dot(np.transpose(mu[1]-mu[0]))
    b = -0.5*(mu[1]+mu[0]).dot(beta)+math.log10(N1/N_1)
    
    for i in test_features:
        predicted_labels.append(sign(i,beta,b))
    return np.array(predicted_labels)

def sign(x,beta,b):
    x = x.dot(beta)+b
    if x > -b:
        return 1
    else:
        return -1
    
training_labels = correct_labels(y_train)
training_prediction = predict_lda(mu, covmat, p, training_features, training_labels)
print("Training Error:", np.mean(training_prediction != training_labels))

test_features = features2d(X_test)
test_labels = correct_labels(y_test)
test_prediction = predict_lda(mu, covmat, p, test_features, test_labels)
print("Test Error:", np.mean(test_prediction != test_labels))



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse


def plot_decision_regions(features, labels, mu, covmat, p):
    # Define the grid
    x_min, x_max = features[:, 0].min() - 1, features[:, 0].max() + 1
    y_min, y_max = features[:, 1].min() - 1, features[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))
    grid = np.c_[xx.ravel(), yy.ravel()]

    # Compute the LDA predictions for the grid
    Z = predict_lda(mu, covmat, p, grid)
    Z = Z.reshape(xx.shape)

    # Plot the decision regions
    plt.contourf(xx, yy, Z, cmap=plt.cm.RdBu, alpha=.8)

    # Plot the training data
    plt.scatter(features[:, 0], features[:, 1], c=labels, cmap=plt.cm.RdBu, edgecolors='black')

    # Plot the cluster shapes
    for i, (m, c) in enumerate(zip(mu, covmat)):
        eigenvalues, eigenvectors = np.linalg.eig(c)
        std_devs = np.sqrt(eigenvalues)
        angle = np.degrees(np.arctan2(*eigenvectors[0][::-1]))
        ellipse = Ellipse(xy=m, width=2 * std_devs[0], height=2 * std_devs[1], angle=angle, alpha=0.3)
        plt.gca().add_artist(ellipse)
        ellipse.set_facecolor('none')
        if i == 0:
            ellipse.set_edgecolor('red')
        else:
            ellipse.set_edgecolor('blue')

        # Plot the cluster axes
        for j, std_dev in enumerate(std_devs):
            if eigenvectors[j][0] == 0:
                angle = np.pi / 2
            else:
                angle = np.arctan(eigenvectors[j][1] / eigenvectors[j][0])
            x, y = m + std_dev * np.array([np.cos(angle), np.sin(angle)])
            plt.plot([m[0], x], [m[1], y], '-', color='black')

    # Add a legend
    plt.legend(['Class 3', 'Class 9', 'Cluster 3', 'Cluster 9'])

    # Show the plot
    plt.show()


mu, covmat, p = fit_lda(train_features, y_train)
try:
    plot_decision_regions(train_features, y_train, mu, covmat, p)
except:
    print("")

In [ ]:
# Problem 3.4

import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import KFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

digits = load_digits()
X = digits.data
y = digits.target
y[y == 3] = -1
y[y == 9] = 1

kf = KFold(n_splits=10, shuffle=True, random_state=42)
test_errors = []
sklearn_test_errors = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    mu, covmat, p = fit_lda(X_train, y_train)
    y_pred = predict_lda(mu, covmat, p, X_test)
    test_error = np.mean(y_pred != y_test)
    test_errors.append(test_error)

    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train, y_train)
    sklearn_y_pred = lda.predict(X_test)
    sklearn_test_error = np.mean(sklearn_y_pred != y_test)
    sklearn_test_errors.append(sklearn_test_error)

print("Our LDA test error: ", np.mean(test_errors))
print("Sklearn LDA test error: ", np.mean(sklearn_test_errors))